<a href="https://colab.research.google.com/github/rajeshvalluri/vidstab/blob/main/VidStab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [105]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from timeit import default_timer as timer

In [106]:
start_time = timer()

**Helper Function to detect blur in frames**

In [107]:
def detect_blur_fft(image, size=60, thresh=10, vis=False):
	# grab the dimensions of the image and use the dimensions to
	# derive the center (x, y)-coordinates
	(h, w) = image.shape
	(cX, cY) = (int(w / 2.0), int(h / 2.0))

	# compute the FFT to find the frequency transform, then shift
	# the zero frequency component (i.e., DC component located at
	# the top-left corner) to the center where it will be more
	# easy to analyze
	fft = np.fft.fft2(image)
	fftShift = np.fft.fftshift(fft)

	# check to see if we are visualizing our output
	if vis:
		# compute the magnitude spectrum of the transform
		magnitude = 20 * np.log(np.abs(fftShift))

		# display the original input image
		(fig, ax) = plt.subplots(1, 2, )
		ax[0].imshow(image, cmap="gray")
		ax[0].set_title("Input")
		ax[0].set_xticks([])
		ax[0].set_yticks([])

		# display the magnitude image
		ax[1].imshow(magnitude, cmap="gray")
		ax[1].set_title("Magnitude Spectrum")
		ax[1].set_xticks([])
		ax[1].set_yticks([])

		# show our plots
		plt.show()

	# zero-out the center of the FFT shift (i.e., remove low
	# frequencies), apply the inverse shift such that the DC
	# component once again becomes the top-left, and then apply
	# the inverse FFT
	fftShift[cY - size:cY + size, cX - size:cX + size] = 0
	fftShift = np.fft.ifftshift(fftShift)
	recon = np.fft.ifft2(fftShift)

	# compute the magnitude spectrum of the reconstructed image,
	# then compute the mean of the magnitude values
	magnitude = 20 * np.log(np.abs(recon))
	mean = np.mean(magnitude)

	# the image will be considered "blurry" if the mean value of the
	# magnitudes is less than the threshold value
	return (mean, mean <= thresh)

**Connect to google drive to load video files**

In [108]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [109]:
cap = cv2.VideoCapture('/content/gdrive/MyDrive/vidstab/CarVideo.mp4')
#get the basic information about the video, like the frame rate(fps), frame_count height and the width
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [9]:
print(cap.get(3),cap.get(4))

1280.0 720.0


In [11]:
#print details acquired above for sanity checks
print("Frame Count:", frame_count, "Width :", width,"Height :",height," at a frame rate of:",fps)

Frame Count: 714 Width : 1280 Height : 720  at a frame rate of: 7.9466778657196935


In [110]:
#setting up a format and profile for output video
#start by defining four character code for encoding
fourcc = cv2.VideoWriter_fourcc(*'MJPG') #MJPG gives the highest resolution image/video

output_video = cv2.VideoWriter('/content/gdrive/MyDrive/vidstab/CarVideo_output.mp4', fourcc,fps, (width, height))

In [111]:
#once the basics have been done, time to start going through each frame and detect motion
#read a frame
_, prev = cap.read()
print(prev.shape)
#convert the frames to grayscale for easy computations 3-channel info will be flattned to a single channel one
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
print(prev_gray[719])

(720, 1280, 3)
[ 46  51  59 ...  86  68 129]


In [ ]:
print(prev)

In [112]:
#start the process of identifying the right features to track, and estimate motion between frames
transforms = np.zeros((frame_count-1,3), np.float32) # 3 deltas to track x, y and z axis movements
# we only need frame_count -1 number of transformations because the last frame doesn't need anymore transformations
print(transforms.shape)

(713, 3)


In [113]:
#use the blur_detector helper method to filter out blurry frames
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
blurry_frames = []
blurry_idx = []
for i in range(frame_count -2):
  #read the next frame
  success, curr = cap.read()
  if (not success): # no more frames to read or read failed
    break 
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
  #if (detect_blur_fft(curr_gray)[1]):
  #  print('Blurry Frame')
  #np.asarray(blurry_frames,detect_blur_fft(curr_gray))
  mean_thresh, blurry = detect_blur_fft(curr_gray,thresh=25)
  if (blurry == True):
    print(mean_thresh,blurry)
    blurry_frames.append(curr)
    blurry_idx.append(i)
    #print("Frame no.", i,"is Blurry")
    #(fig, ax) = plt.subplots(1, 1)
    #ax.imshow(curr)
    #print(curr.shape)
  #plt.show()


24.7043704714652 True
24.843227079297783 True
24.79946042342459 True
24.82582139670966 True
24.773751749710424 True
24.635941800958996 True
24.965761077090576 True
24.559599540960868 True
24.15098778001376 True
23.365388959623896 True
23.74459279939049 True
23.902943455802994 True
23.684824659119904 True
22.989447642512175 True
22.5984281475603 True
21.50498453432441 True
23.358729191390417 True
24.0186890174136 True
23.230402250399152 True
22.861249706946396 True
22.734735451333247 True
23.18418380753202 True
22.421725177114975 True
22.75423690247001 True
22.92154277372427 True
22.352759429691186 True
22.16115522393123 True
22.954498252739494 True
22.73647401335291 True
22.665763259965104 True
22.630760187262716 True
22.662505179687283 True
22.397019129509037 True
23.415235135617753 True
22.459465890054748 True
23.084905597228204 True
22.733209369586717 True
23.542849546046664 True
23.518177821306967 True
23.570826279292977 True
23.245400312638395 True
23.572070899600686 True
23.79390

In [114]:
print(len(blurry_frames),blurry_idx)

59 [500, 501, 502, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 556, 557, 558, 559, 560, 561, 562, 563]


In [ ]:
print(blurry_frames)

In [115]:
#use CV2's goodFeaturesToTrack method to get features
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
for i in range(frame_count -2):
  prev_points = cv2.goodFeaturesToTrack(prev_gray,
                                        maxCorners=200,
                                        qualityLevel=0.01,
                                        minDistance=30,
                                        blockSize=3)
  #read the next frame
  success, curr = cap.read()
  if (not success): # no more frames to read or read failed
    break 
  mean_thresh, blurry = detect_blur_fft(curr_gray,thresh=25)
  if (blurry == True):
    continue
  curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
  #use CV2's opticalflowpyrLK ( Pyramid  Lucas-Kanade  ) method to compute optical flow between current and previous frames
  curr_points, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray,prev_points, None )
  #print ( prev_points.shape, curr_points.shape)
  #print(status.shape)
  idx = np.where( status ==1)[0]
  #culling the previous points and current points arrays to only use the valid ones
  prev_points = prev_points[idx]
  curr_points = curr_points[idx]
  matrix, _ = cv2.estimateAffinePartial2D(prev_points, curr_points)
  print (matrix.shape)
  x_movement = matrix[0][2]
  y_movement = matrix[1][2]
  angle_change = np.arctan2(matrix[1][0],matrix[0][0])
  transforms[i] = [x_movement,y_movement,angle_change]
  #get next frame
  prev_gray = curr_gray
  print("Frame : ", i , "/" , frame_count, " - Tracked points: ", len(prev_points))

(2, 3)
Frame :  0 / 714  - Tracked points:  200
(2, 3)
Frame :  1 / 714  - Tracked points:  179
(2, 3)
Frame :  2 / 714  - Tracked points:  200
(2, 3)
Frame :  3 / 714  - Tracked points:  189
(2, 3)
Frame :  4 / 714  - Tracked points:  200
(2, 3)
Frame :  5 / 714  - Tracked points:  199
(2, 3)
Frame :  6 / 714  - Tracked points:  197
(2, 3)
Frame :  7 / 714  - Tracked points:  200
(2, 3)
Frame :  8 / 714  - Tracked points:  199
(2, 3)
Frame :  9 / 714  - Tracked points:  200
(2, 3)
Frame :  10 / 714  - Tracked points:  200
(2, 3)
Frame :  11 / 714  - Tracked points:  200
(2, 3)
Frame :  12 / 714  - Tracked points:  196
(2, 3)
Frame :  13 / 714  - Tracked points:  193
(2, 3)
Frame :  14 / 714  - Tracked points:  198
(2, 3)
Frame :  15 / 714  - Tracked points:  200
(2, 3)
Frame :  16 / 714  - Tracked points:  200
(2, 3)
Frame :  17 / 714  - Tracked points:  200
(2, 3)
Frame :  18 / 714  - Tracked points:  189
(2, 3)
Frame :  19 / 714  - Tracked points:  196
(2, 3)
Frame :  20 / 714  - Tr

In [116]:
#caculate trajectory between frames
trajectory = np.cumsum(transforms,axis=0)

In [117]:
def movingAverage(curve, radius): 
  window_size = 2 * radius + 1
  # Define the filter 
  f = np.ones(window_size)/window_size 
  # Add padding to the boundaries 
  curve_pad = np.lib.pad(curve, (radius, radius), 'edge') 
  # Apply convolution 
  curve_smoothed = np.convolve(curve_pad, f, mode='same') 
  # Remove padding 
  curve_smoothed = curve_smoothed[radius:-radius]
  # return smoothed curve
  return curve_smoothed 

In [118]:
def smooth(trajectory): 
  smoothed_trajectory = np.copy(trajectory) 
  SMOOTHING_RADIUS = 20
  # Filter the x, y and angle curves
  for i in range(3):
    smoothed_trajectory[:,i] = movingAverage(trajectory[:,i], radius=SMOOTHING_RADIUS)

  return smoothed_trajectory

In [119]:
# Compute trajectory using cumulative sum of transformations
smoothed_trajectory = smooth(trajectory)
difference = smoothed_trajectory - trajectory
transforms_smooth = transforms + difference

In [120]:
# Reset stream to first frame 
cap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
 
# Write n_frames-1 transformed frames
for i in range(frame_count-2):
  # Read next frame
  success, frame = cap.read() 
  if not success:
    break
  mean_thresh, blurry = detect_blur_fft(curr_gray,thresh=25)
  if (blurry == True):
    continue
  # Extract transformations from the new transformation array
  dx = transforms_smooth[i,0]
  dy = transforms_smooth[i,1]
  da = transforms_smooth[i,2]

  # Reconstruct transformation matrix accordingly to new values
  m = np.zeros((2,3), np.float32)
  m[0,0] = np.cos(da)
  m[0,1] = -np.sin(da)
  m[1,0] = np.sin(da)
  m[1,1] = np.cos(da)
  m[0,2] = dx
  m[1,2] = dy

  # Apply affine wrapping to the given frame
  frame_stabilized = cv2.warpAffine(frame, m, (width,height))

  # Fix border artifacts
  frame_stabilized = fixBorder(frame_stabilized) 

  # Write the frame to the file
  frame_out = cv2.hconcat([frame, frame_stabilized])
  #print(frame_out.shape[0],frame_out.shape[1])

  # If the image is too big, resize it.
  if(frame_out.shape[1] > 1280): 
    frame_out = cv2.resize(frame_out, (int(frame_out.shape[1]/2), int(frame_out.shape[0])));
  print("Frame number:", i, frame_out.shape)
  #cv2_imshow(frame_out)
  #cv2.waitKey(10)
  print(frame_out.shape)
  try:
     output_video.write(frame_out)
  except e:
    print("An error has occured:", e)
output_video.release()
print('Processing finished')

Processing finished


In [121]:
end_time = timer()
print('Total Elapsed time =:',end_time-start_time)

Total Elapsed time =: 287.38303801099937


In [58]:
def fixBorder(frame):
  s = frame.shape
  # Scale the image 4% without moving the center
  T = cv2.getRotationMatrix2D((s[1]/2, s[0]/2), 0, 1.04)
  frame = cv2.warpAffine(frame, T, (s[1], s[0]))
  return frame